In [1]:
from data_manipulation import*
from data_preprocessing import*
import pandas as pd
from pathlib import Path
from fm_analycts_parametrs import DEF_STATS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# _df = fm_create_dataframe(path='data/italia_novo_2024.html')

p = Path.cwd()/'data'/'raw'/'Divisoes' 
arquivos = [f for f in p.iterdir() if f.is_file()]



# _df = fm_create_dataframe(path='data/SERIEC_2024.html')

In [ ]:
df_arg = pd.read_html(r'C:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Argentina_2023.html', encoding='utf-8', decimal='.')
df_arg = df_arg[0]
df_arg.dropna(how='all', inplace=True)

In [2]:
_df_concat = pd.DataFrame()
for arquivo in arquivos:
    print(arquivo)
    _df = fm_create_dataframe(path=str(arquivo))
    
    _df_concat = pd.concat([_df_concat, _df], ignore_index=True)
    

c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Alemanha_2024.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Alemanha_2025.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Alemanha_2026.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Alemanha_2027.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Alemanha_2028.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Argentina_2023.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Argentina_2024.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Argentina_2025.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Argentina_2026.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Argentina_2027.html
c:\Users\mjsa2\Documents\GitHub\fm_analise_de_dados\data\raw\Divisoes\Brasil_2023.html
c:\Users\mjsa2\Doc

In [3]:
df = _df_concat.copy()

In [10]:
sorted(df.columns.to_list())

['adef_c_p90',
 'adef_t_p90',
 'alivios_p90',
 'altura',
 'aof_p90',
 'ass',
 'ass_p90',
 'ass_per_aof',
 'aval_cria',
 'aval_def',
 'aval_fin',
 'bloqueios_p90',
 'cab_dec_p90',
 'cab_g_p100',
 'cab_g_p90',
 'cab_p_p90',
 'chutes_gol_p90',
 'chutes_p90',
 'clube',
 'coef',
 'conv_p100',
 'conv_penal_p100',
 'cruz_c_p100',
 'cruz_c_p90',
 'cruz_t_p90',
 'des_c_p100',
 'des_dec_p90',
 'des_g_p90',
 'des_t_p90',
 'divisao',
 'duel_g_p90',
 'duel_t_p90',
 'erros_decisivos_p90',
 'faltas_sofridas_p90',
 'final_contrato',
 'fintas_p90',
 'gc_per_aof',
 'gk_def_desv_p90',
 'gk_def_dif_p90',
 'gk_def_p90',
 'gk_def_segu_p90',
 'gk_gsof_p90',
 'gk_pen_def_p100',
 'gk_sg',
 'gk_xG_def',
 'gk_xG_def_p90',
 'gols',
 'grandes_chances',
 'grandes_chances_p90',
 'id',
 'id_temporada',
 'idade',
 'int_p90',
 'jg_ar_t_p90',
 'melhor_pe',
 'minutos',
 'motm',
 'nac',
 'nome',
 'nota_med',
 'npG',
 'npG_ae',
 'npG_p90',
 'npG_per_aof',
 'np_chutes',
 'np_chutes_gol',
 'np_chutes_gol_p100',
 'np_chutes_g

In [ ]:
print(df['adef_t_p90'].describe())

In [11]:
columns = ['id_temporada','nome', 'idade','posicao_analise','clube', 'divisao','rtg_jg_ar','rtg_duel','rtg_des','rtg_rec_bola','rtg_adef',
           'nota_med','passe_c_p100','passe_t_p90','passe_c_p90','passe_prog_p90','aval_def']

In [19]:
df[df['aval_def'].notna() & (df['id_temporada'] == 2028)][columns].sort_values(by=['aval_def'],ascending=False, ignore_index=True)[:20]

,id_temporada,nome,idade,posicao_analise,clube,divisao,rtg_jg_ar,rtg_duel,rtg_des,rtg_rec_bola,rtg_adef,nota_med,passe_c_p100,passe_t_p90,passe_c_p90,passe_prog_p90,aval_def
0,2028,Rúben Dias,31.0,[Zagueiro],Man City,Premier League,0.78,0.59,0.81,1.27,1.01,7.15,95.0,70.46,66.61,4.66,0.96
1,2028,Giorgio Scalvini,24.0,[Zagueiro],Liverpool,Premier League,0.80,0.61,0.86,1.23,1.02,7.16,95.0,70.61,66.78,4.96,0.96
2,2028,Kim Min-Jae,31.0,[Zagueiro],Bayern München,Bundesliga,0.73,0.60,0.82,1.33,0.99,7.21,93.0,69.72,64.99,4.99,0.95
3,2028,Marcão,31.0,"[Zagueiro, Volante]",Sevilla,LALIGA EA SPORTS,0.70,0.66,0.93,1.18,0.98,6.87,95.0,68.99,65.73,4.74,0.92
4,2028,Leny Yoro,22.0,[Zagueiro],Man Utd,Premier League,0.79,0.57,0.88,1.11,1.01,7.01,94.0,64.23,60.22,5.03,0.92
5,2028,Cameron Carter-Vickers,30.0,[Zagueiro],Real Sociedad,LALIGA EA SPORTS,0.71,0.61,0.87,1.17,0.98,7.04,94.0,70.15,65.66,4.09,0.91
6,2028,Eduardo Quaresma,26.0,"[Lateral-Direito, Zagueiro, Ala-Direito, Volante]",Osasuna,LALIGA EA SPORTS,0.73,0.53,0.75,1.24,0.97,6.82,91.0,68.05,61.63,4.68,0.91
7,2028,Cristian Romero,30.0,[Zagueiro],Tottenham Hotspur,Premier League,0.78,0.56,0.71,1.15,1.01,6.91,94.0,68.81,64.43,4.45,0.91
8,2028,António Silva,24.0,[Zagueiro],Benfica,Liga Portugal Betclic,0.71,0.57,0.78,1.23,0.96,7.17,94.0,67.67,63.51,4.96,0.91
9,2028,Pau Torres,31.0,"[Zagueiro, Volante]",Aston Villa,Premier League,0.79,0.60,0.85,1.07,1.00,7.11,94.0,69.09,65.16,5.01,0.91


In [ ]:
# df['divisao'].unique().tolist()

sorted(df.columns.to_list())

In [ ]:
df['aval_def_adj'] = df['aval_def']*df['coef']
df['rtg_jg_ar_adj'] = df['rtg_jg_ar']*df['coef']
df['rtg_des_adj'] = df['rtg_des']*df['coef']
df['rtg_adef_adj'] = df['rtg_adef']*df['coef']
df['rtg_duel_adj'] = df['rtg_adef']*df['coef']

p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj1'] = (((     (df['rtg_jg_ar_adj']*df['jg_ar_t_p90']*p1) +
                                        df['rtg_duel_adj']*p2 + 
                                        (df['rtg_des_adj']*df['des_t_p90'])*p3 + 
                                        (df['poss_g_p90']/df['adef_t_p90'])*p4+
                                        df['rtg_adef_adj']*p5
                                        ))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj2'] = ((((df['rtg_jg_ar_adj']*p1) + df['rtg_duel_adj']*p2 + (df['rtg_des_adj'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef_adj']*p5))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj3'] = ((((df['rtg_jg_ar']*p1) + df['rtg_duel']*p2 + (df['rtg_des'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef']*p5))).round(2)
df['aval_def_adj4'] = (df['aval_def_adj3']*df['coef']).round(2)

In [ ]:
df_zagueiros = df[df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]
df_zagueiros = df_zagueiros[df_zagueiros['divisao'] == 'Serie C NOW Girone C']
df_zagueiros['aval_def_adj4_pct'] = (df_zagueiros['aval_def_adj4'].rank(pct=True)).round(2)

In [ ]:
df_zagueiros[['nome', 'idade','clube', 'divisao','preco_min', 'posicao_analise', 'aval_def', 'aval_def_adj','aval_def_adj1','aval_def_adj2','aval_def_adj3','aval_def_adj4','aval_def_adj4_pct']].sort_values(by=['aval_def_adj3'],ascending=False, ignore_index=True)

In [ ]:
df_zagueiros['aval_def_adj4'].describe()

In [ ]:
div_med = _df.groupby('divisao')['aval_def'].mean()
div_desv = _df.groupby('divisao')['aval_def'].std()

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

_df_zagueiros['aval_def_adj'] = _df_zagueiros.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_adef_adj'] = _df_zagueiros.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_des_adj'] = _df_zagueiros.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_duel_adj'] = _df_zagueiros.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_jg_ar_adj'] = _df_zagueiros.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

_df_zagueiros['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df_zagueiros['aval_def_adj1'] = (((     (_df_zagueiros['rtg_jg_ar_adj']*_df_zagueiros['jg_ar_t_p90']*p1) +
                                        _df_zagueiros['rtg_duel_adj']*p2 + 
                                        (_df_zagueiros['rtg_des_adj']*_df_zagueiros['des_t_p90'])*p3 + 
                                        (_df_zagueiros['poss_g_p90']/_df_zagueiros['adef_t_p90'])*p4+
                                        _df_zagueiros['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df_zagueiros['aval_def_adj1'] = _df_zagueiros['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df_zagueiros['aval_def_adj2'] = (_df_zagueiros['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

print(_df_zagueiros.groupby('divisao')['rtg_adef'].mean())
print(_df_zagueiros.groupby('divisao')['rtg_adef'].std())

_df_zagueiros[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df_zagueiros[(_df_zagueiros['divisao'].str.contains('Serie C NOW')) & (_df_zagueiros['minutos'] >= 900) & (_df_zagueiros['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'minutos',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def'],ascending=False)

In [ ]:
_df['aval_def_adj'] = _df.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_adef_adj'] = _df.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_des_adj'] = _df.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_duel_adj'] = _df.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_jg_ar_adj'] = _df.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
_df['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df['aval_def_adj1'] = (((     (_df['rtg_jg_ar_adj']*_df['jg_ar_t_p90']*p1) +
                                        _df['rtg_duel_adj']*p2 + 
                                        (_df['rtg_des_adj']*_df['des_t_p90'])*p3 + 
                                        (_df['poss_g_p90']/_df['adef_t_p90'])*p4+
                                        _df['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df['aval_def_adj1'] = _df['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df['aval_def_adj2'] = (_df['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

In [ ]:
_df[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df[(_df['divisao'] == 'Serie A TIM') & (_df['minutos'] >= 900) & (_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def_adj2'],ascending=False)